#### Set the input parameters

In [4]:
center_lat =       -4.171376 # latitude of center point, in decimal degrees
center_lon =  27.735308 # longitude of center point, in decimal degrees
radius = 10 # radius of flower of life, in kilometres
divisions = 6 # creates flower of life by dividing the 360 degree circle into 6 equal parts
rotation = 0 # optional: add a tilt to the grid. I do not recommend tilting your grids
round_to_decimal_place = 2 # i do not recommend changing this. it is used to filter out duplicate coordinates (small values means less filtering)
max_points = 100 # the number of points you want the grid to be. ideally keep this below 500

#### Logic for calculating distances between points and creating pattern

In [6]:
from math import asin, atan2, cos, degrees, radians, sin
import numpy as np
import pandas as pd
from google.colab import files


def get_point_at_distance(lat1, lon1, d, bearing, R=6371):
    """
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2),)



In [9]:

results_df = pd.DataFrame({'lat':[center_lat],
                              'lon':[center_lon]})

seen = []

seen.append([round(center_lat, round_to_decimal_place), round(center_lon,round_to_decimal_place)])

seen_center = []


for i in range(0,100):

  if len(results_df) >= max_points:
    break

  else:


    #display('results_df at start', results_df)

    #print('length of results', len(results_df))

    for row, k in results_df[i:].iterrows():
      #print(k)

      tmp_center_lat = float(k['lat'])
      tmp_center_lon = float(k['lon'])

      # tmp_check_seen_center = [round(tmp_result_lat, round_to_decimal_place), round(tmp_result_lon,round_to_decimal_place)]
      # #print(tmp_check_seen_center)
      # if tmp_check_seen_center in seen_center:
      #     #print('seen center coordinate')
      #     break
      # else:
      #   seen_center.append(tmp_check_seen)



      for j in range(0, 360, int(360/divisions)):
        bearing = j + rotation
        tmp_result_lat, tmp_result_lon = get_point_at_distance(tmp_center_lat, tmp_center_lon, radius, bearing)
        #print('tmp center', tmp_center_lat, ',', tmp_center_lon)
        #print('distance',radius)
        #print('bearing','bearing')

        # tmp_coord = [tmp_result_lat, tmp_result_lon]

        df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat],
                              'lon':[tmp_result_lon]})

        tmp_check_seen = [round(tmp_result_lat, round_to_decimal_place), round(tmp_result_lon,round_to_decimal_place)]


        #display('results_df after dedupe', results_df)

        if len(results_df) <= max_points:

          #print('number', len(results_df))
          #display('df_tmp_coord', df_tmp_coord)
          if tmp_check_seen in seen:
          #print('seen coordinate')
            continue
          else:
            seen.append(tmp_check_seen)

            results_df = pd.concat([results_df, df_tmp_coord])
            #display('results_df after tmp',results_df)

            results_df.loc[results_df.round(round_to_decimal_place).drop_duplicates().index]





          # continue

          # df_tmp_coord = pd.DataFrame(tmp_coord)
          # df_tmp_coord.columns = ['lat','lon']
          # results_df = pd.concat(results_df, df_tmp_coord)

          #print(tmp_coord)

        else:
          break

# print(len(results_df))

In [8]:
# print('latitude'+','+'longitude')
# print(str(r['lat'])+','+str(r['lon']))

#### Creation of Output file. Import it into Google Earth by doing File -> Import and clicking on output.txt

In [10]:
 # pastes the resulting coordinates into a text file to import into Google Earth
output_text = ''
output_text += 'latitude'+','+'longitude'
output_text += '\n'
for i,r in results_df.iterrows():
  tmp_pt = str(r['lat'])+','+str(r['lon'])+'\n'
  output_text += tmp_pt

f = open("output.txt", "w")
f.write(output_text)
files.download('output.txt')
f.close()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>